In [2]:
''' 
DB에서 지표 계산에 필요한 데이터 불러오기 
'''
import sqlite3
import pandas as pd


conn = sqlite3.connect('Database/S&P500.db')
query = "SELECT * FROM daily_stock_price;"
chunks = []
for chunk in pd.read_sql_query(query, conn, chunksize=10000):
    chunks.append(chunk)
daily_stock_price = pd.concat(chunks, ignore_index=True)
query = "SELECT * FROM stock"
tickers = pd.read_sql_query(query, conn)['symbol'].unique()
conn.close()

In [3]:
''' 
대부분의 단기 투자 지표 계산 후 DB에 삽입 
'''
import warnings
import importlib
import pandas as pd
import ta.momentum
import ta.volume
import ta.volatility
import ta.trend
import Custom.CustomizedAnalysis as ca


conn = sqlite3.connect('Database/S&P500.db')
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
importlib.reload(ca)


# 종목 별로 지표계산
for index, ticker in enumerate(tickers):
    if ticker != 'SCHW':
        continue
    data = daily_stock_price.loc[daily_stock_price['symbol'] == ticker].copy()
    
    # 데이터가 없는 종목은 건너뜀
    if len(data) == 0:
        continue
    
    # 데이터 정리
    data['date'] = pd.to_datetime(data['date'])
    data = data.sort_values(by = 'date', ascending=True)
    data = data.reset_index(drop=True)


    ''' 모멘텀 지표 (Momentum Indicator) '''
    # Awesome Oscillator
    data['awesome_oscillator'] = ta.momentum.awesome_oscillator(high=data['high'], low=data['low'])
    data['awesome_oscillator_shift'] = data['awesome_oscillator'].shift(1)
    data['awesome_oscillator_indicator'] = data.apply(ca.awesome_oscillator_indicator, axis=1) # GC
    data.pop('awesome_oscillator_shift')

    # KAMA (Kaufman’s Adaptive Moving Average)
    data['kama'] = ta.momentum.kama(close=data['close'])
    data['kama_close_kama_diff'] = data['close'] - data['kama'] # GC
    data['kama_close_kama_diff_shift'] = data['kama_close_kama_diff'].shift(1)
    data['kama_indicator'] = data.apply(ca.kama_indicator, axis=1) # GC
    data.pop('kama_close_kama_diff_shift')

    # PPO (Percentage Price Oscillator)
    ppo = ta.momentum.PercentagePriceOscillator(close=data['close'])
    data['ppo'] = ppo.ppo()
    data['ppo_signal'] = ppo.ppo_signal()
    data['ppo_hist'] = ppo.ppo_hist() # GC
    data['ppo_hist_shift'] = data['ppo_hist'].shift(1)
    data['ppo_indicator'] = data.apply(ca.ppo_indicator, axis=1) # GC
    data.pop('ppo_hist_shift')

    # PVO (Percentage Volume Oscillator)
    pvo = ta.momentum.PercentageVolumeOscillator(volume=data['volume'])
    data['pvo'] = pvo.pvo()
    data['pvo_signal'] = pvo.pvo_signal()
    data['pvo_hist'] = pvo.pvo_hist() # GC
    data['pvo_hist_shift'] = data['pvo_hist'].shift(1)
    data['pvo_indicator'] = data.apply(ca.pvo_indicator, axis=1) # GC
    data.pop('pvo_hist_shift')

    # ROC (Rate of Change)
    data['roc'] = ta.momentum.roc(close=data['close'])

    # RSI (Relative Strength Index)
    data['rsi'] = ta.momentum.rsi(close=data['close']) 
    data['rsi_shift'] = data['rsi'].shift(1)
    data['rsi_indicator'] = data.apply(ca.rsi_indicator, axis=1) # GC
    data.pop('rsi_shift')

    # Stochastic RSI Oscillator
    stoch_rsi = ta.momentum.StochRSIIndicator(close=data['close'])
    data['stochrsi_k'] = stoch_rsi.stochrsi()
    data['stochrsi_d1'] = stoch_rsi.stochrsi_k()
    data['stochrsi_d2'] = stoch_rsi.stochrsi_d()
    data['stochrsi_k_d1_diff'] = data['stochrsi_k'] - data['stochrsi_d1'] # GC
    data['stochrsi_k_d1_diff_shift'] = data['stochrsi_k_d1_diff'].shift(1)
    data['stochrsi_indicator1'] = data.apply(ca.stochrsi_indicator1, axis=1) # GC
    data.pop('stochrsi_k_d1_diff_shift')
    data['stochrsi_k_d2_diff'] = data['stochrsi_k'] - data['stochrsi_d2'] # GC
    data['stochrsi_k_d2_diff_shift'] = data['stochrsi_k_d2_diff'].shift(1)
    data['stochrsi_indicator2'] = data.apply(ca.stochrsi_indicator2, axis=1) # GC
    data.pop('stochrsi_k_d2_diff_shift')

    # Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(close=data['close'], high=data['high'], low=data['low'])
    data['stoch_k'] = stoch.stoch()
    data['stoch_d'] = stoch.stoch_signal()
    data['stoch_k_d_diff'] = data['stoch_k'] - data['stoch_d'] # GC
    data['stoch_k_d_diff_shift'] = data['stoch_k_d_diff'].shift(1)
    data['stoch_indicator'] = data.apply(ca.stoch_indicator, axis=1) # GC
    data.pop('stoch_k_d_diff_shift')

    # TSI (True Strength Index)
    data['tsi'] = ta.momentum.tsi(close=data['close'])
    data['tsi_signal'] = ta.trend.ema_indicator(close=data['tsi'], window=12) # GC
    data['tsi_tsi_signal_diff'] = data['tsi'] - data['tsi_signal'] # GC
    data['tsi_tsi_signal_diff_shift'] = data['tsi_tsi_signal_diff'].shift(1)
    data['tsi_indicator'] = data.apply(ca.tsi_indicator, axis=1) # GC
    data.pop('tsi_tsi_signal_diff_shift')

    # Ultimate Oscillator
    data['ultimate_oscillator'] = ta.momentum.ultimate_oscillator(high=data['high'], low=data['low'], close=data['close'])
    data['ultimate_oscillator_shift'] = data['ultimate_oscillator'].shift(1)
    data['ultimate_oscillator_indicator'] = data.apply(ca.ultimate_oscillator_indicator, axis=1) # GC
    data.pop('ultimate_oscillator_shift')

    # Williams %R
    data['williams_r'] = ta.momentum.williams_r(high=data['high'], low=data['low'], close=data['close'])
    data['williams_r_shift'] = data['williams_r'].shift(1)
    data['williams_r_indicator'] = data.apply(ca.williams_r_indicator, axis=1) # GC
    data.pop('williams_r_shift')
    

    ''' 거래량 지표 (Volume Indicator) '''
    # ADI (Accumulation/Distribution Index)
    data['adi'] = ta.volume.acc_dist_index(high=data['high'], low=data['low'], close=data['close'], volume=data['volume'])
    data['adi_signal'] = ta.trend.ema_indicator(close=data['adi'], window=12) #GC
    data['adi_adi_signal_diff'] = data['adi'] - data['adi_signal'] # GC
    data['adi_adi_signal_diff_shift'] = data['adi_adi_signal_diff'].shift(1)
    data['adi_indicator'] = data.apply(ca.adi_indicator, axis=1) # GC
    data.pop('adi_adi_signal_diff_shift')
    
    # CMF (Chaikin Money Flow)
    data['cmf'] = ta.volume.chaikin_money_flow(high=data['high'], low=data['low'], close=data['close'], volume=data['volume'])
    data['cmf_shift'] = data['cmf'].shift(1)
    data['cmf_indicator'] = data.apply(ca.cmf_indicator, axis=1) # GC
    data.pop('cmf_shift')
    
    # EoM, EMV (Ease of movement)
    ease_of_movement = ta.volume.EaseOfMovementIndicator(high=data['high'], low=data['low'], volume=data['volume'])
    data['eom'] = ease_of_movement.ease_of_movement()
    data['eom_sma'] = ease_of_movement.sma_ease_of_movement()
    data['eom_sma_shift'] = data['eom_sma'].shift(1)
    data['eom_indicator'] = data.apply(ca.eom_sma_indicator, axis=1)
    data.pop('eom_sma_shift')
    
    # FI (Force Index)
    data['fi'] = ta.volume.force_index(close=data['close'], volume=data['volume'])
    data['fi_signal'] = ta.trend.ema_indicator(close=data['fi'], window=12)
    data['fi_signal_shift'] = data['fi_signal'].shift(1)
    data['fi_indicator'] = data.apply(ca.fi_indicator, axis = 1)
    data.pop('fi_signal_shift')

    # MFI (Money Flow Index)
    data['mfi'] = ta.volume.money_flow_index(high=data['high'], low=data['low'], close=data['close'], volume=data['volume'])
    
    # NVI (Negative Volume Index)
    data['nvi'] = ta.volume.negative_volume_index(close=data['close'], volume=data['volume'])
    data['nvi_signal_short'] = ta.trend.ema_indicator(close=data['nvi'], window=12) # GC
    data['nvi_signal_long'] = ta.trend.ema_indicator(close=data['nvi'], window=255) # GC
    data['nvi_short_long_diff'] = data['nvi_signal_short'] - data['nvi_signal_long'] # GC
    data['nvi_short_long_diff_shift'] = data['nvi_short_long_diff'].shift(1)
    data['nvi_indicator'] = data.apply(ca.nvi_indicator, axis=1) # GC
    data.pop('nvi_short_long_diff_shift')
    
    # OBV (On-Balance Volume)
    data['obv'] = ta.volume.on_balance_volume(close=data['close'], volume=data['volume'])
    data['obv_signal'] = ta.trend.ema_indicator(close=data['obv'], window=12)
    data['obv_obv_signal_diff'] = data['obv'] - data['obv_signal'] # GC
    data['obv_obv_signal_diff_shift'] = data['obv_obv_signal_diff'].shift(1)
    data['obv_indicator'] = data.apply(ca.obv_indicator, axis=1) # GC
    data.pop('obv_obv_signal_diff_shift')
    
    # VPT (Volume-price trend)
    data['vpt'] = ta.volume.volume_price_trend(close=data['close'], volume=data['volume'])
    data['vpt_signal'] = ta.trend.ema_indicator(close=data['vpt'], window=12) # GC
    data['vpt_vpt_signal_diff'] = data['vpt'] - data['vpt_signal'] # GC
    data['vpt_vpt_signal_diff_shift'] = data['vpt_vpt_signal_diff'].shift(1)
    data['vpt_indicator'] = data.apply(ca.vpt_indicator, axis=1) # GC
    data.pop('vpt_vpt_signal_diff_shift')
    
    # VWAP (Volume Weighted Average Price)
    data['vwap'] = ta.volume.volume_weighted_average_price(high=data['high'], low=data['low'], close=data['close'], volume=data['volume'])
    data['vwap_close_vwap_diff'] = data['close'] - data['vwap'] # GC
    

    ''' 변동성 지표 (Volatility Indicator) '''
    # ATR (Average True Range)
    data['atr'] = ta.volatility.average_true_range(high=data['high'], low=data['low'], close=data['close'])
    
    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(close=data['close'])
    data['bollinger_hband'] = bollinger.bollinger_hband()
    data['bollinger_mband'] = bollinger.bollinger_mavg()
    data['bollinger_lband'] = bollinger.bollinger_lband()
    data['bollinger_hband_indicator'] = bollinger.bollinger_hband_indicator() # GC
    data['bollinger_lband_indicator'] = bollinger.bollinger_lband_indicator() # GC
    data['bollinger_pband'] = bollinger.bollinger_pband()
    data['bollinger_wband'] = bollinger.bollinger_wband()

    # Donchian Channels
    donchian = ta.volatility.DonchianChannel(high=data['high'], low=data['low'], close=data['close'])
    data['donchian_channel_hband'] = donchian.donchian_channel_hband()
    data['donchian_channel_mband'] = donchian.donchian_channel_mband()
    data['donchian_channel_lband'] = donchian.donchian_channel_lband()
    data['donchian_channel_hband_indicator'] = data['close'] >= data['donchian_channel_hband'] # GC
    data['donchian_channel_lband_indicator'] = data['close'] <= data['donchian_channel_lband'] # GC
    data['donchian_channel_pband'] = donchian.donchian_channel_pband()
    data['donchian_channel_wband'] = donchian.donchian_channel_wband()

    # Keltner Channel
    keltner = ta.volatility.KeltnerChannel(high=data['high'], low=data['low'], close=data['close'])
    data['keltner_channel_hband'] = keltner.keltner_channel_hband()
    data['keltner_channel_mband'] = keltner.keltner_channel_mband()
    data['keltner_channel_lband'] = keltner.keltner_channel_lband()
    data['keltner_channel_hband_indicator'] = keltner.keltner_channel_hband_indicator() # GC
    data['keltner_channel_lband_indicator'] = keltner.keltner_channel_lband_indicator() # GC
    data['keltner_channel_pband'] = keltner.keltner_channel_pband()
    data['keltner_channel_wband'] = keltner.keltner_channel_wband()
    
    # Ulcer Index
    data['ulcer_index'] = ta.volatility.ulcer_index(close=data['close'])


    ''' 추세지표 (Trend Indicators) '''
    # 단순이동평균 SMA
    data['sma_5'] = ta.trend.sma_indicator(data['close'], window=5)
    data['sma_10'] = ta.trend.sma_indicator(data['close'], window=10)
    data['sma_20'] = ta.trend.sma_indicator(data['close'], window=20)
    data['sma_50'] = ta.trend.sma_indicator(data['close'], window=50)
    data['sma_60'] = ta.trend.sma_indicator(data['close'], window=60)
    data['sma_120'] = ta.trend.sma_indicator(data['close'], window=120)
    data['sma_200'] = ta.trend.sma_indicator(data['close'], window=200)

    # 가중이동평균 WMA
    data['wma_5'] = ta.trend.wma_indicator(data['close'], window=5)
    data['wma_10'] = ta.trend.wma_indicator(data['close'], window=10)
    data['wma_20'] = ta.trend.wma_indicator(data['close'], window=20)
    data['wma_50'] = ta.trend.wma_indicator(data['close'], window=50)
    data['wma_60'] = ta.trend.wma_indicator(data['close'], window=60)
    data['wma_120'] = ta.trend.wma_indicator(data['close'], window=120)
    data['wma_200'] = ta.trend.wma_indicator(data['close'], window=200)

    # 지수이동평균 EMA
    data['ema_5'] = ta.trend.ema_indicator(data['close'], window=5)
    data['ema_10'] = ta.trend.ema_indicator(data['close'], window=10)
    data['ema_20'] = ta.trend.ema_indicator(data['close'], window=20)
    data['ema_50'] = ta.trend.ema_indicator(data['close'], window=50)
    data['ema_60'] = ta.trend.ema_indicator(data['close'], window=60)
    data['ema_120'] = ta.trend.ema_indicator(data['close'], window=120)
    data['ema_200'] = ta.trend.ema_indicator(data['close'], window=200)
    
    # ADX (Average Directional Movement Index)
    adx = ta.trend.ADXIndicator(high=data['high'], low=data['low'], close=data['close'])
    data['adx'] = adx.adx()
    data['adx_pos'] = adx.adx_pos()
    data['adx_neg'] = adx.adx_neg()
    data['adx_pos_neg_diff'] = adx.adx_pos() - adx.adx_neg() # GC
    
    # Aroon Indicator
    aroon = ta.trend.AroonIndicator(high=data['high'], low=data['low'])
    data['aroon_up'] = aroon.aroon_up()
    data['aroon_down'] = aroon.aroon_down()
    data['aroon_up_down_diff'] = aroon.aroon_indicator() # GC
    data['aroon_up_down_diff_shift'] = data['aroon_up_down_diff'].shift(1)
    data['aroon_indicator'] = data.apply(ca.aroon_indicator, axis=1) # GC
    data.pop('aroon_up_down_diff_shift')

    # CCI (Commodity Channel Index)
    data['cci'] = ta.trend.cci(high=data['high'], low=data['low'], close=data['close'])
    
    # DPO (Detrended Price Oscillator)
    data['dpo'] = ta.trend.dpo(close=data['close'])
    data['dpo_shift'] = data['dpo'].shift(1)
    data['dpo_indicator'] = data.apply(ca.dpo_indicator, axis=1) # GC
    data.pop('dpo_shift')

    # 일목균형표 (Ichimoku Kinkō Hyō, Ichimoku)
    ichimoku = ta.trend.IchimokuIndicator(high=data['high'], low=data['low'])
    data['ichimoku_a'] = ichimoku.ichimoku_a()
    data['ichimoku_b'] = ichimoku.ichimoku_b()
    data['ichimoku_conversion_line'] = ichimoku.ichimoku_conversion_line()
    data['ichimoku_base_line'] = ichimoku.ichimoku_base_line()
    data['ichimoku_close_position'] = data.apply(ca.ichimoku_close_position, axis=1) # GC
    data['ichimoku_conversion_base_diff'] = ichimoku.ichimoku_conversion_line() - ichimoku.ichimoku_base_line() # GC
    data['ichimoku_conversion_base_diff_shift'] = data['ichimoku_conversion_base_diff'].shift(1)
    data['ichimoku_conversion_base_indicator'] = data.apply(ca.ichimoku_conversion_base_indicator, axis=1) # GC
    data.pop('ichimoku_conversion_base_diff_shift')

    # KST Oscillator (KST Signal)
    kst = ta.trend.KSTIndicator(close=data['close'])
    data['kst'] = kst.kst()
    data['kst_signal'] = kst.kst_sig()
    data['kst_diff'] = kst.kst_diff() # GC
    data['kst_shift'] = data['kst'].shift(1)
    data['kst_indicator1'] = data.apply(ca.kst_indicator1, axis=1) # GC
    data.pop('kst_shift')
    data['kst_diff_shift'] = data['kst_diff'].shift(1)
    data['kst_indicator2'] = data.apply(ca.kst_indicator2, axis=1) # GC
    data.pop('kst_diff_shift')

    # MACD (Moving Average Convergence Divergence)
    macd = ta.trend.MACD(data['close'])
    data['macd'] = macd.macd()
    data['macd_signal'] = macd.macd_signal()
    data['macd_diff'] = macd.macd_diff() # GC
    data['macd_shift'] = data['macd'].shift(1)
    data['macd_indicator1'] = data.apply(ca.macd_indicator1, axis=1) # GC
    data.pop('macd_shift')
    data['macd_diff_shift'] = data['macd_diff'].shift(1)
    data['macd_indicator2'] = data.apply(ca.macd_indicator2, axis=1) # GC
    data.pop('macd_diff_shift')

    # MI (Mass Index)
    data['mi'] = ta.trend.mass_index(high=data['high'], low=data['low'])

    # Parabolic SAR (Parabolic Stop and Reverse)
    psar = ta.trend.PSARIndicator(high=data['high'], low=data['low'], close=data['close'])
    data['psar'] = psar.psar()
    data['psar_up'] = psar.psar_up()
    data['psar_down'] = psar.psar_down()
    data['psar_up_indicator'] = psar.psar_up_indicator() # GC
    data['psar_down_indicator'] = psar.psar_down_indicator() # GC
    data['psar_close_psar_diff'] = data['close'] - data['psar'] # GC
    data['psar_indicator'] = data.apply(ca.psar_indicator, axis=1) # GC
    data.pop('psar_up_indicator')
    data.pop('psar_down_indicator')

    # STC (Schaff Trend Cycle)
    data['stc'] = ta.trend.stc(close=data['close'])
    data['stc_shift'] = data['stc'].shift(1)
    data['stc_indicator'] = data.apply(ca.stc_indicator, axis=1) # GC
    data.pop('stc_shift')

    # TRIX (Triple Exponential Average)
    data['trix'] = ta.trend.trix(close=data['close'])
    data['trix_shift'] = data['trix'].shift(1)
    data['trix_indicator1'] = data.apply(ca.trix_indicator1, axis=1)
    data.pop('trix_shift')
    data['trix_signal'] = ta.trend.ema_indicator(close=data['trix'], window=12) # GC
    data['trix_trix_signal_diff'] = data['trix'] - data['trix_signal'] # GC
    data['trix_trix_signal_diff_shift'] = data['trix_trix_signal_diff'].shift(1)
    data['trix_indicator2'] = data.apply(ca.trix_indicator2, axis=1) # GC
    data.pop('trix_trix_signal_diff_shift')
    
    # VI (Vortex Indicator)
    vi = ta.trend.VortexIndicator(high=data['high'], low=data['low'], close=data['close'])
    data['vi_pos'] = vi.vortex_indicator_pos()
    data['vi_neg'] = vi.vortex_indicator_neg()
    data['vi_pos_neg_diff'] = vi.vortex_indicator_diff() # GC
    data['vi_pos_neg_diff_shift'] = data['vi_pos_neg_diff'].shift(1)
    data['vi_indicator'] = data.apply(ca.vi_indicator, axis=1) # GC
    data.pop('vi_pos_neg_diff_shift')


    ''' 기타 지표 (Others Indicator)'''
    # 없음
    
    
    ''' 임의 생성 지표 '''
    # CMO (Chande Momentum Oscillator)
    data['cmo'] = ca.cmo(close=data['close'])
    data['cmo_shift'] = data['cmo'].shift(1)
    data['cmo_indicator'] = data.apply(ca.cmo_indicator, axis=1) # GC
    data.pop('cmo_shift')
    
    # Pivot Point
    data['pivot_point_1'] = (data['high'] + data['low'] + data['close']) / 3
    data['pivot_point_2'] = (data['open'] + data['high'] + data['low'] + data['close']) / 4
    
    # 캔들스틱 차트 (Candlestick chart)
    data['doji'] = data.apply(ca.is_doji, axis=1)  # 도지 (Doji) 패턴
    data['hammer'] = data.apply(ca.is_hammer, axis=1)  # 해머 (Hammer)
    data['inverted_hammer'] = data.apply(ca.is_inverted_hammer, axis=1)  # 역해머 (Inverted Hammer)
    data['morning_star'] = [ca.is_morning_star(data, i) for i in range(len(data))]  # 유성형 (Morning Star)
    data['triple_top'] = [ca.is_triple_top(data, i) for i in range(len(data))]  # 삼중천장 (Triple Top)
    data['triple_tottom'] = [ca.is_triple_bottom(data, i) for i in range(len(data))]  # 삼중바닥 (Triple Bottom)
    data['red_marubozu'] = data.apply(ca.is_red_marubozu, axis=1)  # 붉은색 마루보즈 (Red Marubozu)
    data['green_marubozu'] = data.apply(ca.is_green_marubozu, axis=1)  # 녹색 마루보즈 (Green Marubozu)
    data['three_black_crows'] = [ca.is_three_black_crows(data, i) for i in range(len(data))]  # 흑삼병 (Three Black Crows)
    data['three_white_soldiers'] = [ca.is_three_white_soldiers(data, i) for i in range(len(data))]  # 적삼병 (Three White Soldiers)
    
    
    # ''' 현재 주가 대비 n거래일 후 주가 변동률 (1 ~ 21일) '''
    # for i in range(1, 7 * 3 + 1):
    #     data[f'change_after_{i}td'] = ((data['close'].shift(-i) - data['close']) / data['close']) * 100


    # ''' 현재 주가 대비 n거래일 후까지의 이동평균에 대한 주가 변동률 (1 ~ 14일) '''
    # for i in range(1, 7 * 2 + 1):
    #     data[f'change_after_{i}td_ma'] = ((data['close'].rolling(window=i).mean().shift(-i) - data['close']) / data['close']) * 100


    # # DB에 저장
    # data.to_sql('daily_stock_indicator', conn, if_exists='append', index=False)
    # print(f"Ticker : {ticker} - Saved to DB {round(((index + 1) / len(tickers)) * 100, 1)}%")

conn.close()

Ticker : MMM - Saved to DB 0.2%
Ticker : AOS - Saved to DB 0.4%
Ticker : ABT - Saved to DB 0.6%
Ticker : ABBV - Saved to DB 0.8%
Ticker : ACN - Saved to DB 1.0%
Ticker : ADBE - Saved to DB 1.2%
Ticker : AMD - Saved to DB 1.4%
Ticker : AES - Saved to DB 1.6%
Ticker : AFL - Saved to DB 1.8%
Ticker : A - Saved to DB 2.0%
Ticker : APD - Saved to DB 2.2%
Ticker : ABNB - Saved to DB 2.4%
Ticker : AKAM - Saved to DB 2.6%
Ticker : ALB - Saved to DB 2.8%
Ticker : ARE - Saved to DB 3.0%
Ticker : ALGN - Saved to DB 3.2%
Ticker : ALLE - Saved to DB 3.4%
Ticker : LNT - Saved to DB 3.6%
Ticker : ALL - Saved to DB 3.8%
Ticker : GOOGL - Saved to DB 4.0%
Ticker : GOOG - Saved to DB 4.2%
Ticker : MO - Saved to DB 4.4%
Ticker : AMZN - Saved to DB 4.6%
Ticker : AMCR - Saved to DB 4.8%
Ticker : AMTM - Saved to DB 5.0%
Ticker : AEE - Saved to DB 5.2%
Ticker : AEP - Saved to DB 5.4%
Ticker : AXP - Saved to DB 5.6%
Ticker : AIG - Saved to DB 5.8%
Ticker : AMT - Saved to DB 6.0%
Ticker : AWK - Saved to DB 6.2%

In [4]:
'''
DB의 데이터 확인
'''
import sqlite3
import pandas as pd


conn = sqlite3.connect('Database/S&P500.db')
query = "SELECT * FROM daily_stock_indicator;"
chunks = []
for chunk in pd.read_sql_query(query, conn, chunksize=10000):
    chunks.append(chunk)
conn.close()
daily_stock_indicator = pd.concat(chunks, ignore_index=True)
display(daily_stock_indicator)

,symbol,gics_sector,date,adj_close,close,high,low,open,volume,awesome_oscillator,...,doji,hammer,inverted_hammer,morning_star,triple_top,triple_tottom,red_marubozu,green_marubozu,three_black_crows,three_white_soldiers
0,MMM,Industrials,2010-01-04 00:00:00,43.783855,69.414719,69.774246,69.122070,69.473244,3640265,NaN,...,0,0,0,0,0,0,0,0,0,0
1,MMM,Industrials,2010-01-05 00:00:00,43.509621,68.979935,69.590302,68.311035,69.230766,3405012,NaN,...,0,0,0,0,0,0,0,0,0,0
2,MMM,Industrials,2010-01-06 00:00:00,44.126678,69.958191,70.735786,69.824417,70.133781,6301126,NaN,...,0,0,0,0,0,0,0,0,1,0
3,MMM,Industrials,2010-01-07 00:00:00,44.158325,70.008362,70.033447,68.662209,69.665550,5346240,NaN,...,0,1,0,1,0,0,0,0,0,0
4,MMM,Industrials,2010-01-08 00:00:00,44.469482,70.501671,70.501671,69.648827,69.974915,4073337,NaN,...,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784607,ZTS,Health Care,2024-12-11 00:00:00,177.169998,177.169998,178.380005,175.809998,176.259995,1782400,-0.589912,...,0,0,0,0,1,1,0,0,0,0
1784608,ZTS,Health Care,2024-12-12 00:00:00,178.839996,178.839996,179.699997,176.559998,176.949997,1936000,0.188294,...,0,0,0,1,1,1,0,0,0,0
1784609,ZTS,Health Care,2024-12-13 00:00:00,178.179993,178.179993,181.850006,176.630005,178.979996,1650300,0.887647,...,0,0,0,0,1,1,0,0,0,0
1784610,ZTS,Health Care,2024-12-16 00:00:00,175.809998,175.809998,179.289993,175.789993,177.990005,1938800,1.037881,...,0,0,0,0,1,1,0,0,0,0
